In [ ]:
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
%reload_ext nb_black


<div align="center">

# EDS-TeVa DEMO


<img src="https://aphp.github.io/edsteva/latest/assets/logo/edsteva_logo_demo.svg" alt="EDS-TeVa">

</div>

EDS-TeVa can generate synthetic data that mimics OMOP data structure and the behavior of real hospital data. It is here to help you to test the different functionalities. It can also helps to test your custom [probe](https://aphp.github.io/edsteva/latest/components/probe/) and custom [model](https://aphp.github.io/edsteva/latest/components/model/).


## I. Load the synthetic data

In [ ]:
from edsteva.io import SyntheticData
from datetime import datetime

data = SyntheticData(
    mean_visit=1000,  # Average number of visit per hospital
    t_min=datetime(2012, 1, 1),  # Minimum date of a record
    t_max=datetime(2022, 1, 1),  # Maximum date of a record
    seed=None,  # If int, it set a seed for random number generator
    mode="step",  # It "rect", it will generate data with rectangle like deployment
).generate()

In [ ]:
print(data.available_tables)  # See available tables
data.visit_detail.head()

## II. Administrative records modeling

### II.1 Compute the Visit Probe

The [``VisitProbe``](https://aphp.github.io/edsteva/latest/reference/probes/visit/#edsteva.probes.visit.VisitProbe) computes $c_{visit}(t)$ the availability of administrative data related to visits for each care site and each stay type according to time:

$$
c_{visit}(t) = \frac{n_{visit}(t)}{n_{99}}
$$

Where $n_{visit}(t)$ is the number of visits, $n_{99}$ is the $99^{th}$ percentile of visits and $t$ is the month.

<sub>*Note*: If the $99^{th}$ percentile of visits $n_{99}$ is equal to 0, we consider that the completeness predictor $c(t)$ is also equal to 0.</sub>

In [ ]:
from edsteva.probes import VisitProbe

visit = VisitProbe()
visit.compute(
    data,
    stay_types={
        "All": ".*",
        "Urg_Hospit": "urgence|hospitalisés",
        "Consult": "consultation",
    },
    care_site_levels=["Hospital", "Pole", "UF"],
)
visit.save()
visit.predictor.sample(frac=1).head()

### II.2 Fit the Step Fucntion Model

The [``StepFunction``](https://aphp.github.io/edsteva/latest/reference/models/step_function/step_function/#edsteva.models.step_function.step_function.StepFunction) fits a step function $f_{t_0, c_0}(t)$ with coefficients $\Theta = (t_0, c_0)$ on a completeness predictor $c(t)$:

$$
\begin{aligned}
f_{t_0, c_0}(t) & = c_0 \ \mathbb{1}_{t \geq t_0}(t) \\
c(t) & = f_{t_0, c_0}(t) + \epsilon(t)
\end{aligned}
$$

- the characteristic time $t_0$ estimates the time after which the data is available.
- the characteristic value $c_0$ estimates the stabilized routine completeness.

In [ ]:
from edsteva.models.step_function import StepFunction

visit_model = StepFunction()
visit_model.fit(
    probe=visit,
)

## III. Visualize the model

EDS-TeVa library provides dashboards and plots to visualize the temporal evolution of [Probes][probe] along with fitted [Models][model]. Visualization functionalities can be used to explore the database and set thresholds relative to selection criteria.

### III.1 Show interactive dashboard

A **Dashboard** is an interactive [Altair](https://altair-viz.github.io/) chart that lets you visualize variables aggregated by any combination of columns included in the [Probe](https://aphp.github.io/edsteva/latest/components/probe/). In the library, the dashboards are divided into two parts:

- On the top, there is the plot of the aggregated variable of interest.
- On the bottom, there are interactive filters to set. Only the selected data is aggregated to produce the plot on the top.

In [ ]:
from edsteva.viz.dashboards import predictor_dashboard

predictor_dashboard(
    probe=visit,
    fitted_model=visit_model,
    care_site_level="UF",
)

### III.2 Plot the model

This static plot can be exported in png or svg. As it is less interactive, you may specify the filters in the inputs of the functions as follow:

In [ ]:
from edsteva.viz.plots import plot_probe

plot_probe(
    probe=visit,
    fitted_model=visit_model,
    care_site_level="Hospital",
    stay_type="All",
)

## IV. Set the thresholds

The estimates dashboard provides:

- on the top, a representation of the overall deviation from the Model.
- on the bottom, interactive sliders that allows you to vary the thresholds.

One use could be to set the thresholds that keep the most care sites while having an acceptable overall deviation.

In [ ]:
from edsteva.viz.dashboards import estimates_dashboard

estimates_dashboard(
    probe=visit,
    fitted_model=visit_model,
    care_site_level="UF",
)